In [137]:
!pip install efficientnet_pytorch
!!pip install mlxtend --upgrade --no-deps
#importing required modules
import gdown
import zipfile
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torchsummary import summary
from torchvision import datasets, transforms as T
from efficientnet_pytorch import EfficientNet
import os
import torch.nn.functional as F
from tqdm.notebook import tqdm
import torch.optim as optim
from PIL import ImageFile
from sklearn.metrics import accuracy_score
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [138]:
#declaring batch size
batch_size = 16
lr=0.001
epochs=10
#applying required transformations on the dataset
img_transforms = {
    'train':
    T.Compose([
        T.Resize(size=(224,224)), 
        T.RandomRotation(degrees=(-20,20)),#ramdomly rotate the image by -/+20
        T.ToTensor(),
        T.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]), 
        ]),

    'valid':
    T.Compose([
        T.Resize(size=(224,224)),
        T.ToTensor(),
        T.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
        ]),

    'test':
    T.Compose([
        T.Resize(size=(224,224)),
        T.ToTensor(),
        T.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
        ]),
     }

# creating Location of data: train, validation, test
train_path="/content/drive/MyDrive/data_dir/Train"
valid_path="/content/drive/MyDrive/data_dir/Valid"
test_path="/content/drive/MyDrive/data_dir/Test"


# creating Datasets to each of  folder created in prev
train_file=datasets.ImageFolder(train_path,transform=img_transforms['train'])
valid_file=datasets.ImageFolder(valid_path,transform=img_transforms['valid'])
test_file=datasets.ImageFolder(test_path,transform=img_transforms['test'])


#Creating loaders for the dataset
loaders_transfer={
    'train':torch.utils.data.DataLoader(train_file,batch_size,shuffle=True),
    'valid':torch.utils.data.DataLoader(valid_file,batch_size,shuffle=True),
    'test': torch.utils.data.DataLoader(test_file,batch_size,shuffle=True)
}

In [139]:
#importing the pretrained EfficientNet model

model = EfficientNet.from_pretrained('efficientnet-b7')

# Freeze weights
for param in model.parameters():
    param.requires_grad = False
in_features = model._fc.in_features


# Defining Dense top layers after the convolutional layers
model._fc = nn.Sequential(
    nn.BatchNorm1d(num_features=in_features),    
    nn.Linear(in_features, 512),
    nn.ReLU(),
    nn.BatchNorm1d(512),
    nn.Linear(512, 128),
    nn.ReLU(),
    nn.BatchNorm1d(num_features=128),
    nn.Dropout(0.4),
    nn.Linear(128, 3),
    )
device= torch.device("cuda" if torch.cuda.is_available() else "cpu")
model =model.to(device)
model

Loaded pretrained weights for efficientnet-b7


EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 64, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(64, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        64, 64, kernel_size=(3, 3), stride=[1, 1], groups=64, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(64, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        64, 16, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        16, 64, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        64, 32, kernel_siz

In [140]:
model.to(device) # move the model to GPU
summary(model,input_size=(3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         ZeroPad2d-1          [-1, 3, 225, 225]               0
Conv2dStaticSamePadding-2         [-1, 64, 112, 112]           1,728
       BatchNorm2d-3         [-1, 64, 112, 112]             128
MemoryEfficientSwish-4         [-1, 64, 112, 112]               0
         ZeroPad2d-5         [-1, 64, 114, 114]               0
Conv2dStaticSamePadding-6         [-1, 64, 112, 112]             576
       BatchNorm2d-7         [-1, 64, 112, 112]             128
MemoryEfficientSwish-8         [-1, 64, 112, 112]               0
          Identity-9             [-1, 64, 1, 1]               0
Conv2dStaticSamePadding-10             [-1, 16, 1, 1]           1,040
MemoryEfficientSwish-11             [-1, 16, 1, 1]               0
         Identity-12             [-1, 16, 1, 1]               0
Conv2dStaticSamePadding-13             [-1, 64, 1, 1]           1,088
         I

In [141]:
def accuracy(y_pred,y_true):
    y_pred = F.softmax(y_pred,dim = 1)
    top_p,top_class = y_pred.topk(1,dim = 1)
    equals = top_class == y_true.view(*top_class.shape)
    return torch.mean(equals.type(torch.FloatTensor))

In [142]:
class efficientNetTrainer():
    
    def __init__(self,criterion = None,optimizer = None,schedular = None):
        
        self.criterion = criterion
        self.optimizer = optimizer
        self.schedular = schedular
    
    def train_batch_loop(self,model,trainloader):
        
        train_loss = 0.0
        train_acc = 0.0
        
        for images,labels in tqdm(trainloader): 
            
            # move the data to CPU
            images = images.to(device)
            labels = labels.to(device)
            
            logits = model(images)
            loss = self.criterion(logits,labels)
            
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            
            train_loss += loss.item()
            train_acc += accuracy(logits,labels)
            
        return train_loss / len(trainloader), train_acc / len(trainloader) 

    
    def valid_batch_loop(self,model,validloader):
        
        valid_loss = 0.0
        valid_acc = 0.0
        
        for images,labels in tqdm(validloader):
            
            # move the data to CPU
            images = images.to(device) 
            labels = labels.to(device)
            
            logits = model(images)
            loss = self.criterion(logits,labels)
            
            valid_loss += loss.item()
            valid_acc += accuracy(logits,labels)
            
        return valid_loss / len(validloader), valid_acc / len(validloader)
            
        
    def fit(self,model,trainloader,validloader,epochs):
        
        valid_min_loss = np.Inf 
        
        for i in range(epochs):
            
            model.train() # this turn on dropout
            avg_train_loss, avg_train_acc = self.train_batch_loop(model,trainloader) ###
            
            model.eval()  # this turns off the dropout lapyer and batch norm
            avg_valid_loss, avg_valid_acc = self.valid_batch_loop(model,validloader) ###
            
            if avg_valid_loss <= valid_min_loss :
                print("Valid_loss decreased {} --> {}".format(valid_min_loss,avg_valid_loss))
                torch.save(model.state_dict(),'/content/efficientNet_potato7.pt')
                valid_min_loss = avg_valid_loss

                
            print("Epoch : {} Train Loss : {:.6f} Train Acc : {:.6f}".format(i+1, avg_train_loss, avg_train_acc))
            print("Epoch : {} Valid Loss : {:.6f} Valid Acc : {:.6f}".format(i+1, avg_valid_loss, avg_valid_acc))

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr =lr)
trainer = efficientNetTrainer(criterion,optimizer)
import time
start = time.time()
trainer.fit(model,loaders_transfer['train'],loaders_transfer['valid'],epochs = epochs)
print("Total time: ", time.time() - start, "seconds")

  0%|          | 0/204 [00:00<?, ?it/s]

  0%|          | 0/27 [00:00<?, ?it/s]

Valid_loss decreased inf --> 0.251156665108822
Epoch : 1 Train Loss : 0.447727 Train Acc : 0.820976
Epoch : 1 Valid Loss : 0.251157 Valid Acc : 0.900926


  0%|          | 0/204 [00:00<?, ?it/s]

  0%|          | 0/27 [00:00<?, ?it/s]

...

Valid_loss decreased 0.18561149305767483 --> 0.17990265179563453
Epoch : 4 Train Loss : 0.308965 Train Acc : 0.879289
Epoch : 4 Valid Loss : 0.179903 Valid Acc : 0.956019


  0%|          | 0/204 [00:00<?, ?it/s]

  0%|          | 0/27 [00:00<?, ?it/s]

Epoch : 5 Train Loss : 0.289915 Train Acc : 0.887663
Epoch : 5 Valid Loss : 0.217449 Valid Acc : 0.923148


  0%|          | 0/204 [00:00<?, ?it/s]

In [ ]:
model.load_state_dict(torch.load('/content/efficientNet_potato7.pt'))
all_preds = torch.tensor([])
all_preds=all_preds.to(device)
for idx,(image,label) in enumerate(test_file):
    image,label = test_file[idx]
    ps = model(image.to(device).unsqueeze(0))
    ps = F.softmax(ps,dim = 1)
    all_preds = torch.cat(
            (all_preds, ps)
            ,dim=0
        )

In [ ]:
targets =[s[1] for s in test_file.samples]
targets=torch.tensor(targets)
targets

In [ ]:
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix

In [ ]:
def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

In [ ]:
pred=get_num_correct(all_preds,targets.to(device))
pred

In [ ]:
accuracy=pred/len(test_file)
accuracy

In [ ]:
cm = confusion_matrix(targets.type(dtype=torch.int32).cpu(), all_preds.argmax(dim=1).type(dtype=torch.int32).cpu())
cm

In [ ]:
labels = ['Early_Blight', 'Healthy', 'Late_Blight']
fig, ax = plot_confusion_matrix(conf_mat=cm,
                                colorbar=True,
                                show_absolute=True,
                                show_normed=True,
                                class_names=labels)
plt.show()